# Preparations

## Libraries

In [ ]:
!pip install prophet
!pip install sktime
!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sktime.performance_metrics.forecasting import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import r2_score

# XGBoost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

from prophet.make_holidays import make_holidays_df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Helper Function

In [ ]:
def train_test_split_by_date(df, split_date="2024-01-01", test_end="2025-01-01"):
  data_raw = df.copy()
  train = data_raw[data_raw.index < split_date]
  test = data_raw[(data_raw.index >= split_date) & (data_raw.index < test_end)]
  return train, test

def error_metrics(real, pred): # Compare predictions and real values
  mae = mean_absolute_error(real, pred)
  mse = mean_squared_error(real, pred)
  mape = mean_absolute_percentage_error(real, pred)
  smape = mean_absolute_percentage_error(real, pred, symmetric=True)
  r2 = r2_score(real, pred)  # Calculate R-squared

  # Print results
  print(f"Mean Absolute Error (MAE): {mae:.2f}")
  print(f"Mean Squared Error (MSE): {mse:.2f}")
  print(f"Mean Absolute Percentage Error (MAPE): {mape*100:.2f}%")
  print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape*100:.2f}%")
  print(f"R-squared (R²): {r2:.4f}")  # Display R-squared

# Station-wide forecasting

### Load Data

In [ ]:
# Station wide
train = pd.read_csv("/content/drive/Shared drives/Time Series/divvy_data/prod/station/divvy_station_train.csv")
test = pd.read_csv("/content/drive/Shared drives/Time Series/divvy_data/prod/station/divvy_station_test.csv")

# Ensure dates are in datetime format
train['ds'] = pd.to_datetime(train['date'])
test['ds'] = pd.to_datetime(test['date'])
train.drop(columns=['date'], inplace=True)
test.drop(columns=['date'], inplace=True)
# test = test[test["ds"]<"2025-01-01"]

# Create a combined dataframe with a marker column
train['is_train'] = True
test['is_train'] = False
combined = pd.concat([train, test])

# # Add Cluster
cluster_df_2023 = pd.read_csv("/content/drive/Shared drives/Time Series/other_data/2023_cluster_df.csv").rename(columns={"cluster": "cluster_2023"})
cluster_df_2022 = pd.read_csv("/content/drive/Shared drives/Time Series/other_data/2022_cluster_df.csv").rename(columns={"cluster": "cluster_2022"})
cluster_df_2021 = pd.read_csv("/content/drive/Shared drives/Time Series/other_data/2021_cluster_df.csv").rename(columns={"cluster": "cluster_2021"})

combined = combined.merge(cluster_df_2023, on="start_station_name", how="left")
combined = combined.merge(cluster_df_2022, on="start_station_name", how="left")
combined = combined.merge(cluster_df_2021, on="start_station_name", how="left")
combined = combined.drop(columns=["Unnamed: 0", "Unnamed: 0_x", "Unnamed: 0_y"])

combined['cluster_2023'] = combined['cluster_2023'].fillna(0)
combined['cluster_2022'] = combined['cluster_2022'].fillna(0)
combined['cluster_2021'] = combined['cluster_2021'].fillna(0)
combined = pd.get_dummies(combined, columns=['cluster_2021','cluster_2022','cluster_2023'],drop_first=True)
combined
combined.head()

<ipython-input-19-3da75b2cb11c>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("/content/drive/Shared drives/Time Series/divvy_data/prod/station/divvy_station_train.csv")


,start_station_name,start_station_id,total_rides,temp_min_c,rain_sum_mm,snowfall_sum_cm,month,dayofweek,year,ds,is_train
0,63rd St Beach,101,2,-5.5,0.0,0.0,1,2,2020,2020-01-01,True
1,Orleans St & Hubbard St,636,5,-5.5,0.0,0.0,1,2,2020,2020-01-01,True
2,Orleans St & Elm St,23,3,-5.5,0.0,0.0,1,2,2020,2020-01-01,True
3,Orleans St & Chestnut St (NEXT Apts),620,5,-5.5,0.0,0.0,1,2,2020,2020-01-01,True
4,Ogden Ave & Roosevelt Rd,434,2,-5.5,0.0,0.0,1,2,2020,2020-01-01,True


## Training

In [ ]:
for lag in [1, 2, 7, 30, 60, 90, 365]:
    combined[f'y_lag{lag}'] = combined.groupby('start_station_name')['total_rides'].shift(lag)

# Feature engineering for XGBoost
for df in [combined]:
    df['day'] = df['ds'].dt.day
    df['month'] = df['ds'].dt.month
    df['year'] = df['ds'].dt.year
    df['weekday'] = df['ds'].dt.weekday
    df['week_of_year'] = df['ds'].dt.isocalendar().week
    df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)
    df['rain_intensity'] = pd.cut(df['rain_sum_mm'], bins=[-1, 0, 5, 20, np.inf], labels=[0, 1, 2, 3])
    df['snow_flag'] = (df['snowfall_sum_cm'] > 0).astype(int)
    df['rolling_avg_7'] = df.groupby('start_station_name')['total_rides'].transform(lambda x: x.shift(1).rolling(window=7, min_periods=1).mean())
    df['rolling_avg_30'] = df.groupby('start_station_name')['total_rides'].transform(lambda x: x.shift(1).rolling(window=30, min_periods=1).mean())
    df['ride_growth'] = df.groupby('start_station_name')['total_rides'].transform(lambda x: x.shift(7).pct_change(periods=7).fillna(0))
    df['rain_intensity'] = df['rain_intensity'].astype(float)
    df['weekend_rain'] = df['is_weekend'] * df['rain_intensity']
    df['cold_no_snow'] = (df['temp_min_c'] < 5) & (df['snow_flag'] == 0)

train = combined[combined['is_train']].drop('is_train', axis=1)
test = combined[~combined['is_train']].drop('is_train', axis=1)
train.drop(columns=['ds', 'start_station_id'], inplace=True)
test.drop(columns=['ds', 'start_station_id'], inplace=True)

# Ensure only common stations exist in test before encoding
test = test[test['start_station_name'].isin(train['start_station_name'].unique())]

# One-hot encoding
train = pd.get_dummies(train, columns=['start_station_name'], drop_first=True)
test = pd.get_dummies(test, columns=['start_station_name'], drop_first=True)

# Align test columns with train columns
test = test.reindex(columns=train.columns, fill_value=0)

# Prepare data for XGBoost
X_train = train.drop(columns=['total_rides'])
y_train = train['total_rides']
X_test = test.drop(columns=['total_rides'])
y_test = test['total_rides']

X_train.head()

<ipython-input-20-20fd3864f52c>:16: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['ride_growth'] = df.groupby('start_station_name')['total_rides'].transform(lambda x: x.shift(7).pct_change(periods=7).fillna(0))


,temp_min_c,rain_sum_mm,snowfall_sum_cm,month,dayofweek,year,y_lag1,y_lag2,y_lag7,y_lag30,...,start_station_name_Woodlawn & 103rd - Olive Harvey Vaccination Site,start_station_name_Woodlawn Ave & 55th St,start_station_name_Woodlawn Ave & 58th St,start_station_name_Woodlawn Ave & 75th St,start_station_name_Woodlawn Ave & Lake Park Ave,start_station_name_Yale Ave & 119th St,start_station_name_Yates Blvd & 75th St,start_station_name_Yates Blvd & 93rd St,start_station_name_Yates Blvd & Exchange Ave,start_station_name_Zapata Academy
0,-5.5,0.0,0.0,1,2,2020,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,-5.5,0.0,0.0,1,2,2020,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,-5.5,0.0,0.0,1,2,2020,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,-5.5,0.0,0.0,1,2,2020,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
4,-5.5,0.0,0.0,1,2,2020,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
best_params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
    'max_depth': 10,
    'n_estimators': 1000,
    'subsample': 0.8
}

# Train the model with the best parameters
best_xgb_reg = XGBRegressor(**best_params, tree_method='hist', device='cuda', random_state=42, verbosity=1)
best_xgb_reg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_xgb_reg.predict(X_test)

# Evaluate the model
error_metrics(y_test, y_pred)

Mean Absolute Error (MAE): 4.79
Mean Squared Error (MSE): 89.87
Mean Absolute Percentage Error (MAPE): 48.48%
Symmetric Mean Absolute Percentage Error (sMAPE): 35.29%
R-squared (R²): 0.8926


In [ ]:
# save
pickle.dump(best_xgb_reg, open("/content/drive/Shared drives/Time Series/Notebooks/Modeling/model_files/stationwide_xgboost_withoutcluster.pkl", "wb"))

### Preprocess

### model training

In [ ]:
best_params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
    'max_depth': 10,
    'n_estimators': 1000,
    'subsample': 0.8
}

# Initialize and train GPU-accelerated model
best_xgb_model = XGBRegressor(**best_params,tree_method="gpu_hist", random_state=42)
best_xgb_model.fit(X_train, y_train)

# Ensure test features match train
X_test = X_test[X_train.columns]

# Make predictions
y_pred = best_xgb_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# R-squared score
r2 = best_xgb_model.score(X_test, y_test)
print(f'R-squared: {r2}')

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:31:06] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:37:51] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:37:51] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.


Mean Squared Error: 47.39549255371094
R-squared: 0.9433770775794983


### Save model

In [ ]:
# save as pickels
import pickle
pickle.dump(best_xgb_model, open("/content/drive/Shared drives/Time Series/Notebooks/Modeling/model_files/stationwide_xgboost_withcluster.pkl", "wb"))

# City-wide forecasting

### Load Data

### XGBoost

#### Training

In [ ]:
def preprocess_tree_models_citywide(rides_citywide,
                  price_flag=True,
                  lags=[1, 7, 30, 90, 365]):
  df = rides_citywide.copy()

  # Convert date column to datetime
  df['date'] = pd.to_datetime(df['date'])

  # Historical Rides
  lags_columns = []
  for lag in lags:
    df[f'y_lag{lag}'] = df['total_rides'].shift(lag)
    lags_columns.append(f'y_lag{lag}')

  # Time Variables
  df['year'] = df['year']-2020
  df = pd.get_dummies(df, columns=['month'], prefix='month')
  df['week_of_year'] = df["date"].dt.isocalendar().week
  df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
  df = pd.get_dummies(df, columns=['dayofweek'], prefix='dayofweek')
  time_columns = ['year', 'week_of_year', 'is_weekend'] \
   + [f"month_{i}" for i in range(1,13)] \
    + [f"dayofweek_{i}" for i in range(7)]

  # Weather
  quantiles_train = df[(df["date"]<"2024-01-01") & (df['rain_sum_mm']>0)]['rain_sum_mm'].quantile([0.5])
  q = quantiles_train.values[0]
  print(f"Rain Boundary:{q}")
  bins = [-np.inf, 0, q, np.inf]  # Include -inf and inf to cover all ranges
  df['rainfall_level'] = pd.cut(df['rain_sum_mm'], bins=bins, labels=["no", "moderate", "heavy"])
  df = pd.get_dummies(df, columns=['rainfall_level'], prefix='rainfall_level')

  df['snowfall_flag'] = (df['snowfall_sum_cm'] > 0).astype(int)

  weather_columns = ["temp_min_c","snowfall_flag"] \
   + [f"rainfall_level_{i}" for i in ["no", "moderate", "heavy"]]

  # Holiday
  holidays = make_holidays_df(list(range(2021,2026)),"US")
  df = pd.merge(left=df, right=holidays,how="left",left_on="date",right_on="ds")
  df["holiday_binary"] = df["holiday"].notna().astype(int)

  # Price
  if price_flag:
    divvy_pricing_history = pd.DataFrame({
      "date": ["2019-07-01", "2022-05-10", "2023-03-29", "2024-02-05"],
      "Annual Membership": [108, 119, 130.9, 143.9],
      "Day Pass": [15, 16.5, 16.5, 18.1],
      "Unlock Fee exists": [0, 1, 1, 1],
      "Member ebike Unlock Fee exists": [0, 1, 0, 0],
    })
    divvy_pricing_history["date"] = pd.to_datetime(divvy_pricing_history["date"])
    df = pd.merge_asof(
      df,
      divvy_pricing_history,
      on='date',
      direction='backward'
    )
    price_columns = divvy_pricing_history.columns.tolist()
    price_columns.remove("date")

  # Supply Side
  supply_columns = ["stations_ma365", "ebike_proportion_ma365"]

  # Selected Columns
  df.set_index("date", inplace=True)
  selected_columns = ["total_rides"] \
   + lags_columns \
    + time_columns \
     + weather_columns \
      + ["holiday_binary"] \
       + supply_columns
  if  price_flag:
    selected_columns = selected_columns + price_columns

  df = df[selected_columns].astype(float)
  df.reset_index(inplace=True)



  df = df[df["date"]>= "2021-01-01"]
  df.dropna(inplace=True)
  return df
for price_flag in [True, False]:
  rides_citywide_model = preprocess_tree_models_citywide(rides_citywide,
                              price_flag=price_flag,
                              lags=[1,2,7,30,90,365])
  rides_citywide_model.set_index("date", inplace=True)
  train, test = train_test_split_by_date(rides_citywide_model)
  x_train = train.drop(columns=['total_rides'])
  y_train = train['total_rides']
  x_test = test.drop(columns=['total_rides'])
  y_test = test['total_rides']

  best_params = {'alpha': 1,
  'colsample_bytree': 0.8,
  'lambda': 1,
  'learning_rate': 0.01,
  'max_depth': 3,
  'n_estimators': 1000,
  'subsample': 0.8}

  # Train the model with the best parameters
  best_xgb_reg = XGBRegressor(**best_params, tree_method='hist', device='cuda', random_state=42, verbosity=1)
  best_xgb_reg.fit(x_train, y_train)

  # Make predictions on the test set
  y_pred = best_xgb_reg.predict(x_test)

  # Evaluate the model
  error_metrics(y_test, y_pred)

  pickle.dump(best_xgb_reg, open(f"/content/drive/Shared drives/Time Series/Notebooks/Modeling/model_files/citywide_xgboost_price{price_flag}.pkl", "wb"))

Rain Boundary:2.5


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:36:55] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Mean Absolute Error (MAE): 1905.39
Mean Squared Error (MSE): 6500352.31
Mean Absolute Percentage Error (MAPE): 18.94%
Symmetric Mean Absolute Percentage Error (sMAPE): 15.71%
R-squared (R²): 0.9124
Rain Boundary:2.5
Mean Absolute Error (MAE): 1891.50
Mean Squared Error (MSE): 6410182.24
Mean Absolute Percentage Error (MAPE): 18.95%
Symmetric Mean Absolute Percentage Error (sMAPE): 15.64%
R-squared (R²): 0.9137


In [ ]:
# City_wide
train_city = pd.read_csv("/content/drive/Shared drives/Time Series/divvy_data/prod/overall/divvy_train.csv")
test_city = pd.read_csv("/content/drive/Shared drives/Time Series/divvy_data/prod/overall/divvy_test.csv")
rides_citywide = pd.concat([train_city, test_city])



<ipython-input-10-becc33299989>:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train_station = pd.read_csv("/content/drive/Shared drives/Time Series/divvy_data/prod/station/divvy_station_train.csv")
